# Cryptocurrency exchange rates to USD ($)

In [ ]:
import json
import os

import ipywidgets as ipw
from IPython.display import display
import pandas as pd

from gamestonk_terminal import api as gst

ipw.HTML(f"<style>{gst.widgets.price_card_stylesheet()}</style>")

In [ ]:
COIN_LIST = (
    "BTC,ETH,USDT,BNB,USDC,XRP,LUNA,ADA,"
    + "SOL,AVAX,DOT,BUSD,DOGE,UST,SHIB,WBTC,"
    + "MATIC,CRO,DAI,LTC,ATOM,NEAR,LINK,BCH,"
    + "UNI,TRX,FTT,ETC,LEO,ALGO,XLM,MANA,"
    + "BTCB,HBAR,EGLD,ICP,SAND,XMR,WAVES,VET,"
    + "APE,FIL,FTM,AXS,THETA,KLAY,XTZ,RUNE"
).split(",")
exchange_rates = {}

In [ ]:
def get_exchange_rate(coin: str, days: int = 2):
    """Get exchange rate for a cryptocurrency agains USD."""
    (_, _, _, _, current_df, _,) = gst.crypto.load(
        coin=coin,
        source="cp",
        days=days,
        vs="usd",
        interval="1day",
        should_load_ta_data=True,
    )
    price_color = "neutral_color"
    if current_df["Close"].iloc[-1] > current_df["Close"].iloc[-2]:
        price_color = "up_color"
    elif current_df["Close"].iloc[-1] < current_df["Close"].iloc[-2]:
        price_color = "down_color"
    price = str(current_df["Close"].iloc[-1])[:7]

    return price, price_color

In [ ]:
def show_rates(btn):
    """Show exchange rates in grid box widget."""
    grid.children = ()
    for coin in COIN_LIST:
        if coin not in exchange_rates.keys():
            exchange_rates[coin] = {"price": None, "color": None}
        try:
            price, price_color = get_exchange_rate(coin=coin)
        except:
            price, price_color = "-------", "neutral_color"
        widgets = list(grid.children)
        widgets.append(
            ipw.HTML(
                gst.widgets.price_card(
                    ticker=coin,
                    price=price,
                    price_color=price_color,
                )
            )
        )
        grid.children = tuple(widgets)

In [ ]:
update_rates = ipw.Button(description="Update rates")
update_rates.on_click(show_rates)
header = ipw.HBox([update_rates])

layout = ipw.Layout(grid_template_columns="1fr " * 8)
grid = ipw.GridBox(
    [],
    layout=layout,
)

display(header, grid)